### This is a test project to predict daily SP500 using AlphaVantage (AV) API for datapull

In [23]:
import numpy as np
import pandas as pd
import os, time, warnings, random, shap, requests
import seaborn as sns
import matplotlib.pyplot as plt
import functools as ft


from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, r2_score, mean_squared_error
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier, XGBRegressor

pd.set_option('display.max_columns', 100)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings('ignore') 

os.chdir('/home/jupyter/projects_gcp_cpu/spx/src')
os.getcwd()

'/home/jupyter/projects_gcp_cpu/spx/src'

Datapull using AV APIs:

In [25]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=SPY&apikey=KBYUCPQEHAG67WNC&datatype=csv&outputsize=full'
# r = requests.get(url)
# data = r.json()
spy = pd.read_csv(url)
display(spy.shape, spy.head())
spy = spy[['timestamp', 'adjusted_close']]
display(spy.shape, spy.head())


(5803, 9)

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2022-11-21,394.64,395.82,392.66,394.59,394.59,51243189,0.0,1.0
1,2022-11-18,397.74,397.81,393.04,396.03,396.03,92922451,0.0,1.0
2,2022-11-17,390.46,394.95,390.14,394.24,394.24,74496255,0.0,1.0
3,2022-11-16,396.78,397.78,394.79,395.45,395.45,68508454,0.0,1.0
4,2022-11-15,401.15,402.31,394.49,398.49,398.49,93194452,0.0,1.0


(5803, 2)

,timestamp,adjusted_close
0,2022-11-21,394.59
1,2022-11-18,396.03
2,2022-11-17,394.24
3,2022-11-16,395.45
4,2022-11-15,398.49


In [26]:
url_1yr = 'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=1year&apikey=KBYUCPQEHAG67WNC&datatype=csv&outputsize=full'
tr1 = pd.read_csv(url_1yr)
tr1.rename(columns={'value':'yield_1y'}, inplace=True)

url_10yr = 'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=10year&apikey=KBYUCPQEHAG67WNC&datatype=csv&outputsize=full'
tr10 = pd.read_csv(url_10yr)
tr10.rename(columns={'value':'yield_10y'}, inplace=True)

url_30yr = 'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=30year&apikey=KBYUCPQEHAG67WNC&datatype=csv&outputsize=full'
tr30 = pd.read_csv(url_30yr)
tr30.rename(columns={'value':'yield_30y'}, inplace=True)

display(tr30)

rates = ft.reduce(lambda left, right: pd.merge(left, right, on='timestamp'), [tr1, tr10, tr30])
display(rates)

,timestamp,yield_30y
0,2022-11-18,3.92
1,2022-11-17,3.89
2,2022-11-16,3.85
3,2022-11-15,3.98
4,2022-11-14,4.07
...,...,...
11934,1977-02-21,.
11935,1977-02-18,7.76
11936,1977-02-17,7.67
11937,1977-02-16,7.67


,timestamp,yield_1y,yield_10y,yield_30y
0,2022-11-18,3.82,3.82,3.92
1,2022-11-17,3.77,3.77,3.89
2,2022-11-16,3.67,3.67,3.85
3,2022-11-15,3.80,3.80,3.98
4,2022-11-14,3.88,3.88,4.07
...,...,...,...,...
11934,1977-02-21,.,.,.
11935,1977-02-18,7.41,7.41,7.76
11936,1977-02-17,7.26,7.26,7.67
11937,1977-02-16,7.34,7.34,7.67


In [29]:
df = pd.merge(spy, rates, on='timestamp')
df.rename(columns = {'adjusted_close':'spx'}, inplace=True)
df.spx = df.spx*10
df.head()

,timestamp,spx,yield_1y,yield_10y,yield_30y
0,2022-11-18,3960.3,3.82,3.82,3.92
1,2022-11-17,3942.4,3.77,3.77,3.89
2,2022-11-16,3954.5,3.67,3.67,3.85
3,2022-11-15,3984.9,3.80,3.80,3.98
4,2022-11-14,3951.2,3.88,3.88,4.07
